## Data Simulation

In [27]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import default_rng
from sklearn.covariance import EmpiricalCovariance

rng = default_rng(123)

In [28]:
def simulate_sigma(d, rho=0.5):
    """
    Simple correlation matrix generator as a stand-in for a vine-generated Σ
        used in Kunzel et al.
    """
    Sigma = rho * np.ones((d, d)) + (1 - rho) * np.eye(d)
    return Sigma

def mu0(x):
    # example control response function
    return x[:, 0]

def mu1(x):
    # example treated response function
    return x[:, 0] + 0.5 * x[:, 1]

def propensity_e(x, alpha=0.5):
    """
    Propensity score function determining probability of treatment = 1
        e(x) = P(W=1|X=x),
        with 'alpha' controlling strength of confounding
        (alpha = 0 is completely random treatment assignment)
    """
    logits = alpha * x[:, 0]
    return 1 / (1 + np.exp(-logits))

def simulate_dataset(N, d=5, alpha=0.5, rho=0.5, rng=None):
    rng = rng or default_rng()
    Sigma = simulate_sigma(d, rho=rho)

    # 1. Generate features X ~ N(0, Σ)
    X = rng.multivariate_normal(mean=np.zeros(d), cov=Sigma, size=N)

    # 2. Generate potential outcomes
    eps0 = rng.normal(0, 1, size=N)
    eps1 = rng.normal(0, 1, size=N)
    Y0 = mu0(X) + eps0
    Y1 = mu1(X) + eps1

    # True treatment effect (CATE)
    tau = Y1 - Y0

    # 3. Generate treatment assignment W ~ Bern(e(X))
    e_x = propensity_e(X, alpha=alpha)
    W = rng.binomial(1, e_x)

    # Generate observed outcome
    Y = np.where(W == 1, Y1, Y0)

    return X, W, Y, Y0, Y1, tau, e_x


### Example train and test data

In [29]:
X_train, W_train, Y_train, Y0_train, Y1_train, tau_train, e_x_train = simulate_dataset(10000, rng=rng)
X_test, W_test, Y_test, Y0_test, Y1_test, tau_test, e_x_test = simulate_dataset(10000, rng=rng)

# Example X-learner

In [30]:
from econml.metalearners import XLearner
from sklearn.linear_model import LinearRegression
import scipy.special

In [31]:
# simple lin reg base model

est = XLearner(models=LinearRegression())
est.fit(Y_train, W_train, X=X_train)

In [32]:
# predicted CATE for train set
tau_hat = est.effect(X_train)

# PEHE := tau_hat - tau
PEHE = (tau_hat - tau_train)**2
print(f"Train PEHE: {PEHE.mean()}")

Train PEHE: 1.9869131682518633


In [33]:
# predicted CATE for test set
tau_hat = est.effect(X_test)

# PEHE := tau_hat - tau
PEHE = (tau_hat - tau_test)**2
print(f"Test PEHE: {PEHE.mean()}")

Test PEHE: 2.041400770917259


# Grid search tuning for base learner

In [34]:
# wrapper for econml Xlearner

class EconMLWrapper:
    def __init__(self, est):
        self.est = est
        self.true_tau = None

    # Required by sklearn for cloning
    def get_params(self, deep=True):
        return {"est": self.est}

    def set_params(self, **params):
        for k, v in params.items():
            setattr(self, k, v)
        return self

    # Fit with special wrapper
    def fit(self, X, y, score_y=None):
        Y = y[:, 0]
        W = y[:, 1]
        self.true_tau = score_y
        self.est.fit(Y, W, X=X)
        return self

    def predict(self, X):
        return self.est.effect(X)

    def score(self, X, y=None):
        preds = self.predict(X)
        return -np.mean((self.true_tau - preds) ** 2)

In [35]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV


base_rf = RandomForestRegressor()

xlearner = XLearner(models=base_rf)

grid_params = {
    "est__models__n_estimators": [50, 100, 200],
    "est__models__max_depth": [3, 5, 8],
    "est__models__min_samples_leaf": [1, 5, 10],
}

y_train_wrapper = np.column_stack((Y_train, W_train))

grid = GridSearchCV(
    estimator=EconMLWrapper(xlearner),
    param_grid=grid_params,
    cv=3,
    n_jobs=-1,
    verbose=1
)

In [36]:
grid.fit(X_train, y_train_wrapper, score_y=tau_train)

best_est = grid.best_estimator_.est
print("\nBest params:", grid.best_params_)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


/Users/gabriellamessenger/Desktop/thesis/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/gabriellamessenger/Desktop/thesis/.venv/lib/python3.13/site-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/gabriellamessenger/Desktop/thesis/.venv/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/Users/gabriellamessenger/Desktop/thesis/.venv/lib/python3.13/site-packages/sklearn/metrics/_scorer.py", line 472, in __call__
    return estimator.score(*args, **kwargs)
           ~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^
  File "/var/folders/yg/yd_rkzv93459x04g3jzsjwcr0000gn/T/ipykern


Best params: {'est__models__max_depth': 3, 'est__models__min_samples_leaf': 1, 'est__models__n_estimators': 50}


In [37]:
tau_hat_train = best_est.effect(X_train)
tau_hat_test = best_est.effect(X_test)

print("\nGrid Search Train PEHE:", pehe(tau_train, tau_hat_train))
print("Grid Search Test PEHE:", pehe(tau_test, tau_hat_test))


Grid Search Train PEHE: 1.2355755279215015
Grid Search Test PEHE: 1.4524457184855648
